In [331]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
data = pd.read_csv("0001.HK.csv", index_col = "Date")

In [23]:
data.index.dayofweek
# data.index
# type(data.index)
# data.index = pd.to_datetime(data.index)

Int64Index([1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
            ...
            3, 4, 0, 1, 2, 3, 4, 0, 1, 2],
           dtype='int64', name='Date', length=4805)

In [239]:
def turning_points(array):
    """ turning_points(array) -> min_indices, max_indices
    Finds the turning points within an 1D array and returns the indices of the minimum and
    maximum turning points in two separate lists.
    """
    idx_max, idx_min = [], []
    if len(array) < 3:
        return idx_min, idx_max

    NEUTRAL, RISING, FALLING = range(3)

    def get_state(a, b):
        if a < b: return RISING
        if a > b: return FALLING
        return NEUTRAL

    ps = get_state(array[0], array[1])
    begin = 1
    for i in range(2, len(array)):
        s = get_state(array[i - 1], array[i])
        if s != NEUTRAL:
            if ps != NEUTRAL and ps != s:
                if s == FALLING:
                    idx_max.append((begin + i - 1) // 2)
                else:
                    idx_min.append((begin + i - 1) // 2)
            begin = i
            ps = s
    return idx_min, idx_max 

def create_turning_point_matrix_for_day_diff(data, day_diff, id_tp_array):
    # shift the turning point array to certain day difference, with the close price of turning point 
    id_tp_array = pd.DataFrame({'max' : [int(i in idx_max) for i in range(len(data))]})
    id_tp_array = id_tp_array.set_index(data.index)
    idx_tp_shift = id_tp_array.shift(day_diff)
    px_shift = data['Adj Close'].shift(day_diff)
    # cal px diff of Tday's px and TP's px
    px_diff = ((data['Adj Close'] - px_shift) / px_shift).mul(idx_tp_shift['max'], fill_value=0)
    
    
    # binning and transform to one hot categorization
    # fibonacci_bins = [-np.inf, -0.764, -0.618, -0.5, -0.382, 0, 0.382, 0.5, 0.618, 0.764, np.inf]
    bins = [-np.inf, -0.2, -0.1, -0.05, 0, 0.05, 0.1, 0.2, np.inf]
    names = ['<-0.2', '-0.2--0.1', '-0.1--0.05', ' -0.05-0', '0-0.05', '0.05-0.1', '0.1-0.2', '>0.2']
    
    px_diff_bin = pd.cut(px_diff, bins, labels=names)
    px_diff_bin = pd.get_dummies(px_diff_bin)
    px_diff_bin[px_diff==0] = 0 # px-diff==0 is matched into 1 bin, it should be zero in all cols

    return px_diff_bin

In [55]:
px = pd.DataFrame({"px": [1,2,3,4,5,6,7,8,6,5,4,3,2,1,2,3,2], "a": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]})

In [242]:
idx_min, idx_max = turning_points(data['Adj Close'])


# plt.plot(data[:100].index, pd.Series(max_array)[:100]*30)
# plt.plot(data['Adj Close'][:100], marker='o', markevery=idx_min)
# plt.plot(data['Adj Close'][:100], marker='o', markevery=idx_max)
# plt.ylabel('some numbers')
# plt.show()

In [329]:
max_matrix2 = create_turning_point_matrix_for_day_diff(data, 2, idx_max)
max_matrix3 = create_turning_point_matrix_for_day_diff(data, 3, idx_max)
max_matrix5 = create_turning_point_matrix_for_day_diff(data, 5, idx_max)
max_matrix10 = create_turning_point_matrix_for_day_diff(data, 10, idx_max)
max_matrix15 = create_turning_point_matrix_for_day_diff(data, 15, idx_max)
    
result_matrix = pd.concat([max_matrix2, max_matrix3, max_matrix5, max_matrix10, max_matrix15], keys=[2,3,5,10,15])
result_matrix.index=result_matrix.index.swaplevel(1,0)
result_matrix = result_matrix.sort_index()
result_matrix
result_matrix.loc['2000-01-13']

<-0.2  -0.2--0.1  -0.1--0.05   -0.05-0  0-0.05  0.05-0.1  \
Date                                                                      
2000-01-13 2       0          0           0         1       0         0   
           3       0          0           0         0       0         0   
           5       0          0           0         0       0         0   
           10      0          0           0         0       0         0   
           15      0          0           0         0       0         0   

               0.1-0.2  >0.2  
Date                          
2000-01-13 2         0     0  
           3         0     0  
           5         0     0  
           10        0     0  
           15        0     0